In [1]:
#import the necessary libraries 
import nltk
import gensim 
from gensim.models import Word2Vec
from fastai.text.all import *
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('averaged_perceptron_tagger')
import re
from nltk import WordNetLemmatizer, pos_tag, word_tokenize
from nltk.corpus import stopwords
from sklearn.cluster import DBSCAN
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
nltk.download('wordnet_ic')
from nltk.tokenize import word_tokenize

import numpy as np
import string
import pandas as pd


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


In [2]:
contexts = pd.read_csv('contexts.csv',index_col=0)
contexts_unique = np.unique(contexts['context'].values)
len(contexts_unique)
chosen_contexts = np.random.choice(contexts_unique, 30, replace=False)

In [3]:
def createVocab( docList):
    vocab = {}
    for doc in docList:
        #print(doc)
        doc= doc.translate(str.maketrans('', '', string.punctuation))
        
        words= word_tokenize(doc.lower())
        for word in words:
            if(word in vocab.keys()):
                vocab[word] = vocab[word] +1
            else:
                vocab[word] =1 
    return vocab

#define a function to tokenize a sentence 
def tokenize_sentence(sentence): 
    tokens = nltk.word_tokenize(sentence) 
    return tokens


def tokenize_sentence2(sentences):
    lower_tokens = [s.lower().split(' ') for s in sentences]
    lower_tokens2 = [[re.sub('[^A-Za-z0-9]+', '', l_) for l_ in l] for l in lower_tokens]
    return(lower_tokens2)

#define a function to preprocess data by breaking up each sentence into individual tokens
def preprocess_data(sentences): 
    tokens_list = []
    for sentence in sentences:
        tokens = tokenize_sentence(sentence)
        tokens_list.append(tokens)
    return tokens_list


def reduce_vocab_by_similarity(model, min_similarity):
    # Initialize the new vocabulary
    model = Word2Vec(sentences, min_count=1)
    #words = list(model.wv.index_to_key) 
    #print(words) # ['Sally', 'ran', 'to', 'the', 'store', 'then','fell','down','and','cut','her','knee']  

    new_vocab = set()

    # Iterate over the words in the model's vocabulary
    for word in model.wv.key_to_index:
        # Initialize the similar words list
        similar_words = []

        # Iterate over the words in the model's vocabulary
        for other_word in model.wv.key_to_index:
            # Calculate the similarity between the two words
            similarity = model.wv.similarity(word, other_word)

            # If the similarity is greater than the minimum similarity
            if similarity > min_similarity:
                # Add the word to the list of similar words
                similar_words.append(other_word)

        # Add the word and its list of similar words to the new vocabulary
        print(word)
        print(similar_words)
        new_vocab.add((word, similar_words))

    # Return the new vocabulary
    return new_vocab

def lemmatize(sentences):
  lemmatized_sentences = []
  # Loop through each sentence
  for sentence in sentences:
    # Initialize WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Tokenize the sentence
    tokens = word_tokenize(sentence)
    # Lemmatize each token
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Join lemmatized tokens
    lemmatized_sentence = ' '.join(lemmatized_tokens)
    # Append to list of lemmatized sentences
    lemmatized_sentences.append(lemmatized_sentence) 
  # Return lemmatized sentences
  return(lemmatized_sentences)

def remove_stop_words(sentences):
  # Initialize WordNet Lemmatizer
  
  # Lemmatize stop words
  
  # Initialize empty array for lemmatized sentences without stop words
  lemmatized_sentences_without_stop_words = []
  # Loop through each sentence
  for sentence in sentences:
    # Tokenize the sentence
    tokens = word_tokenize(sentence)
    # Lemmatize each token
    #lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Filter out stop words
    filtered_tokens = [token for token in tokens if token not in lemmatized_stop_words]
    # Join filtered tokens
    filtered_sentence = ' '.join(filtered_tokens)
    # Append to list of lemmatized sentences without stop words
    lemmatized_sentences_without_stop_words.append(filtered_sentence) 
  # Return lemmatized sentences without stop words
  return lemmatized_sentences_without_stop_words

def extract_POS_tags(sentences):
    """
    This function takes an array of sentences and extracts the parts of speech tags for each word in the sentences.
    
    Parameters:
    sentences (list): An array of sentences
    
    Returns:
    pos_tags (list): An array of parts of speech tags for each word in the sentences
    """
    pos_tags = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        pos_tags.append(nltk.pos_tag([w.lower() for w in words]))
    return pos_tags

def extract_entities(sentences):
    from transformers import pipeline

    model = pipeline('ner', model='bert-base-cased', aggregation_strategy="max")
    #model = pipeline('ner', model='dslim/bert-base-NER', aggregation_strategy="max")
    
    # use the model to tag entities in a given sentence

    predicted_entities = []

    # convert the predicted_entities into the appropriate format for your application 
    # e.g. a list of (entity_name, entity_type) tuples

    for sentence in chosen_contexts:
        predicted_entities.append(model(sentence))
        
    return(predicted_entities)

#useful for running for bert NER tokens
def get_types(text):
    # Import the SpaCy library
    import spacy
    # Load the English language model
    nlp = spacy.load('en_core_web_sm')

    # Parse the text with SpaCy
    doc = nlp(text)
    new_text = []
    
    for token in doc:
        new_text.append([token.text,token.pos_])
    # Return the new text
    return(pd.DataFrame(new_text))

def reduce_vocab_by_similarity(data, min_similarity):
    model = Word2Vec(data, min_count=1)
    # Initialize the new vocabulary
    new_vocab = set()

    # Iterate over the words in the model's vocabulary
    for word in model.wv.key_to_index:
        # Initialize the similar words list
        similar_words = []

        # Iterate over the words in the model's vocabulary
        for other_word in model.wv.key_to_index:
            # Calculate the similarity between the two words
            similarity = model.wv.similarity(word, other_word)

            # If the similarity is greater than the minimum similarity
            if similarity > min_similarity:
                # Add the word to the list of similar words
                similar_words.append(other_word)

        # Convert the similar words list into a tuple
        similar_words_tuple = tuple(similar_words)

        # Add the word and its list of similar words to the new vocabulary
        new_vocab.add((word, similar_words_tuple))

    # Return the new vocabulary
    return new_vocab

def integrate(values,bins_):

    # Group the rows by every two integers and average the values in the groups
    df_grouped = values.groupby(pd.cut(values.index, bins=bins_)).mean()

    # Print the resulting DataFrame
    return(df_grouped)

def derive_centroids(data, labels):
    # Calculate number of clusters
    n_clusters = len(np.unique(labels))
  
    # Create empty centroid array
    centroids = np.zeros((n_clusters, data.shape[1]))
    
    # Get indices of each cluster 
    for i in range(n_clusters):
        indices = np.where(labels == i)[0]
        
        # Calculate mean of each cluster and store it as centroid
        centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
    
    return centroids

def bcss_tss_ratio(X, cluster_centers):

    # Computing Within Cluster Sum of Squares (WCSS)
    wcss = np.sum([np.nansum((x - cluster_centers[i]) ** 2) for i in range(len(cluster_centers)) for x in X[labels == i]])

    # Computing Between Cluster Sum of Squares (BCSS)
    bcss = np.sum([np.nansum((x - center) ** 2) for center in cluster_centers for x in X])

    # Computing Total Sum of Squares (TSS)
    tss = np.sum([np.nansum((x - np.nanmean(X)) ** 2) for x in X])

    # Computing the Ratio of WCSS to TSS
    wcss_tss_ratio = wcss/tss

    # Computing the Ratio of BCSS to TSS
    bcss_tss_ratio = bcss/tss

    #return wcss_tss_ratio, bcss_tss_ratio
    return bcss_tss_ratio



In [4]:
tokenized = tokenize_sentence2(chosen_contexts)
#lower_tokens = [c.lower().split(' ') for c in chosen_contexts]
#lower_tokens2 = [[re.sub('[^A-Za-z0-9]+', '', l_) for l_ in l] for l in lower_tokens]

In [5]:
lemmatizer = WordNetLemmatizer()
lemmatized_stop_words = [lemmatizer.lemmatize(word) for word in stopwords.words('english')]

#lemmas = lemmatize(chosen_contexts)
lemmas = lemmatize([' '.join(t) for t in tokenized])

lemmas_no_stop = remove_stop_words(lemmas)

pos = extract_POS_tags(lemmas_no_stop)

pos_df = [pd.DataFrame(p,columns=['label','type']) for p in pos]

external = []
for p in pos_df:
    values = remove_stop_words(lemmatize(p['label'].values))
    result = p.drop_duplicates(subset='label')['label'].values
    unique_labels = p.groupby('label').count().loc[result].index
    
    inside = re.sub('[^A-Za-z0-9 ]+', '', ' '.join(unique_labels)).replace('  ',' ')
    external.append(inside)
    
pos_lemma_no_stop = remove_stop_words(external)

pos_lemma_no_stop_array = [e.split(' ') for e in pos_lemma_no_stop]

In [6]:
pos_lemma_no_stop_array

[['federal',
  'judge',
  'alex',
  'kozinski',
  'pointed',
  'binding',
  'precedent',
  'know',
  'today',
  'simply',
  'exist',
  'time',
  'constitution',
  'framed',
  'judicial',
  'decision',
  'consistently',
  'accurately',
  'faithfully',
  'reported',
  'side',
  'atlantic',
  'reporter',
  'often',
  'rewrote',
  'failed',
  'publish',
  'disliked',
  'united',
  'kingdom',
  'lacked',
  'coherent',
  'court',
  'hierarchy',
  'prior',
  'end',
  '19th',
  'century',
  'furthermore',
  'english',
  'eighteenth',
  'subscribed',
  'nowobsolete',
  'natural',
  'law',
  'theory',
  'believed',
  'existence',
  'independent',
  'individual',
  'said',
  'saw',
  'merely',
  'declaring',
  'always',
  'theoretically',
  'existed',
  'making',
  'therefore',
  'could',
  'reject',
  'another',
  'opinion',
  'incorrect',
  'statement',
  'way',
  'scientist',
  'regularly',
  'others',
  'conclusion',
  'science'],
 ['presbyterianism',
  'part',
  'reformed',
  'tradition',
  

In [7]:
#model_ = Word2Vec(lower_tokens2 ,min_count=1,workers=4)
model = Word2Vec(pos_lemma_no_stop_array,min_count=1,workers=4)

In [8]:

import itertools

# Recreating array from question
#a = [np.array([[0.00353654]])] * 13

# Make an iterator to yield items of the flattened list and create a list from that iterator
flattened = list(itertools.chain.from_iterable(pos_lemma_no_stop_array))

word2vec_tokens = [model.wv.key_to_index[word] for word in np.unique(flattened)]

word2vec_words_sorted = [model.wv.index_to_key[w] for w in np.sort(word2vec_tokens)]

In [9]:
vectors = [[model.wv.key_to_index[p_] for p_ in p] for p in pos_lemma_no_stop_array]

In [10]:
reduced_vectors = []
for v in vectors:
    reduced_vectors.append(integrate(pd.DataFrame(v),np.max([int(len(v)/10),50])))
    
reduced_vectors_rounded = []
for v in reduced_vectors:
    internal = []
    for v_ in v.values:
        #internal.append(np.round(v_[0],0))
        internal.append((v_[0],0))
    
    reduced_vectors_rounded.append(pd.DataFrame(internal).dropna()[0].values)
    
reduced_vectors_rounded = [[int(v_) for v_ in v] for v in reduced_vectors_rounded]
reduced_phrases = [[model.wv.index_to_key[v_] for v_ in v] for v in reduced_vectors_rounded]

In [11]:
chosen_contexts[0]

"As federal judge Alex Kozinski has pointed out, binding precedent as we know it today simply did not exist at the time the Constitution was framed. Judicial decisions were not consistently, accurately, and faithfully reported on both sides of the Atlantic (reporters often simply rewrote or failed to publish decisions which they disliked), and the United Kingdom lacked a coherent court hierarchy prior to the end of the 19th century. Furthermore, English judges in the eighteenth century subscribed to now-obsolete natural law theories of law, by which law was believed to have an existence independent of what individual judges said. Judges saw themselves as merely declaring the law which had always theoretically existed, and not as making the law. Therefore, a judge could reject another judge's opinion as simply an incorrect statement of the law, in the way that scientists regularly reject each other's conclusions as incorrect statements of the laws of science."

In [12]:


docs=[' '.join(p) for p in reduced_phrases]

vocab = createVocab(docs)

#Compute document term matrix as well idf for each term 

termDict={}

docsTFMat = np.zeros((len(docs),len(vocab)))

docsIdfMat = np.zeros((len(vocab),len(docs)))

docTermDf = pd.DataFrame(docsTFMat ,columns=sorted(vocab.keys()))
docCount=0
for doc in docs:
    doc= doc.translate(str.maketrans('', '', string.punctuation))
    word_synsets = [wordnet.synsets(word) for word in word_tokenize(doc.lower())]
    for word_synset in word_synsets:
        if (word_synset):
            for lemma in word_synset[0].lemmas():
                if(lemma.name() in vocab.keys()):
                    docTermDf[lemma.name()][docCount] = docTermDf[lemma.name()][docCount] +1
          
    docCount = docCount +1

feature_names = sorted(vocab.keys())
docList=np.array(range(0,len(docs),1))

#Computed idf for each word in vocab
idfDict={}

for column in docTermDf.columns:
    idfDict[column]= np.log((len(docs) +1 )/(1+ (docTermDf[column] != 0).sum()))+1
    
#compute tf.idf matrix
docsTfIdfMat = np.zeros((len(docs),len(vocab)))
docTfIdfDf = pd.DataFrame(docsTfIdfMat ,columns=sorted(vocab.keys()))

docCount = 0
for doc in docs:
    for key in idfDict.keys():
        docTfIdfDf[key][docCount] = docTermDf[key][docCount] * idfDict[key]
    docCount = docCount +1 

skDocsTfIdfdf = pd.DataFrame(docTfIdfDf, index=sorted(docList), columns=feature_names)
print(skDocsTfIdfdf)

from sklearn.metrics.pairwise import cosine_similarity

#compute cosine similarity
csim = cosine_similarity(docTfIdfDf, docTfIdfDf)

csimDf = pd.DataFrame(csim,index=sorted(docList),columns=sorted(docList))
csimDf


    10inch       12  12inch       13  1347  1350  169  1700  1786  1798  ...  \
0      0.0  0.00000     0.0  0.00000   0.0   0.0  0.0   0.0   0.0   0.0  ...   
1      0.0  0.00000     0.0  0.00000   0.0   0.0  0.0   0.0   0.0   0.0  ...   
2      0.0  0.00000     0.0  0.00000   0.0   0.0  0.0   0.0   0.0   0.0  ...   
3      0.0  0.00000     0.0  0.00000   0.0   0.0  0.0   0.0   0.0   0.0  ...   
4      0.0  0.00000     0.0  0.00000   0.0   0.0  0.0   0.0   0.0   0.0  ...   
5      0.0  0.00000     0.0  0.00000   0.0   0.0  0.0   0.0   0.0   0.0  ...   
6      0.0  0.00000     0.0  0.00000   0.0   0.0  0.0   0.0   0.0   0.0  ...   
7      0.0  0.00000     0.0  0.00000   0.0   0.0  0.0   0.0   0.0   0.0  ...   
8      0.0  0.00000     0.0  0.00000   0.0   0.0  0.0   0.0   0.0   0.0  ...   
9      0.0  0.00000     0.0  0.00000   0.0   0.0  0.0   0.0   0.0   0.0  ...   
10     0.0  0.00000     0.0  0.00000   0.0   0.0  0.0   0.0   0.0   0.0  ...   
11     0.0  0.00000     0.0  0.00000   0

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.000000,0.100180,0.070529,0.027519,0.000000,0.020047,0.000000,0.035801,0.059704,0.000000,...,0.041667,0.062813,0.119601,0.014148,0.035943,0.000000,0.028445,0.050286,0.039473,0.079697
1,0.100180,1.000000,0.028756,0.020541,0.030407,0.000000,0.025159,0.136864,0.088531,0.029811,...,0.057449,0.081005,0.017455,0.000000,0.030799,0.061716,0.052917,0.096645,0.000000,0.000000
2,0.070529,0.028756,1.000000,0.000000,0.016256,0.011305,0.048491,0.000000,0.042347,0.020474,...,0.000000,0.000000,0.000000,0.021522,0.035174,0.000000,0.000000,0.030635,0.041793,0.000000
3,0.027519,0.020541,0.000000,1.000000,0.031909,0.000000,0.140761,0.000000,0.061496,0.000000,...,0.042968,0.025661,0.023017,0.000000,0.053714,0.040867,0.031997,0.093587,0.038861,0.062513
4,0.000000,0.030407,0.016256,0.031909,1.000000,0.000000,0.014269,0.035326,0.089936,0.072421,...,0.029662,0.082628,0.000000,0.000000,0.023250,0.019606,0.043780,0.025216,0.013758,0.017305
5,0.020047,0.000000,0.011305,0.000000,0.000000,1.000000,0.000000,0.021818,0.008449,0.057310,...,0.026053,0.031770,0.000000,0.000000,0.222489,0.019085,0.029896,0.000000,0.015591,0.034829
6,0.000000,0.025159,0.048491,0.140761,0.014269,0.000000,1.000000,0.000000,0.023282,0.038240,...,0.000000,0.062964,0.055772,0.022627,0.023016,0.033676,0.027867,0.075137,0.093771,0.000000
7,0.035801,0.136864,0.000000,0.000000,0.035326,0.021818,0.000000,1.000000,0.000000,0.000000,...,0.018406,0.033957,0.012194,0.000000,0.000000,0.032285,0.000000,0.000000,0.000000,0.000000
8,0.059704,0.088531,0.042347,0.061496,0.089936,0.008449,0.023282,0.000000,1.000000,0.022957,...,0.051930,0.012074,0.029927,0.019264,0.049151,0.074725,0.019875,0.000000,0.032196,0.015712
9,0.000000,0.029811,0.020474,0.000000,0.072421,0.057310,0.038240,0.000000,0.022957,1.000000,...,0.024869,0.000000,0.062715,0.000000,0.000000,0.018560,0.000000,0.101223,0.017327,0.000000


In [13]:
df_melt = pd.melt(csimDf.reset_index(), id_vars='index', value_vars=csimDf.columns, var_name='x', value_name='value')

In [14]:
print(chosen_contexts[1])

Presbyterianism is a part of the Reformed tradition within Protestantism which traces its origins to the British Isles. Presbyterian churches derive their name from the presbyterian form of church government, which is governed by representative assemblies of elders. Many Reformed churches are organized this way, but the word "Presbyterian," when capitalized, is often applied uniquely to the churches that trace their roots to the Scottish and English churches that bore that name and English political groups that formed during the English Civil War. Presbyterian theology typically emphasizes the sovereignty of God, the authority of the Scriptures, and the necessity of grace through faith in Christ. Presbyterian church government was ensured in Scotland by the Acts of Union in 1707 which created the kingdom of Great Britain. In fact, most Presbyterians found in England can trace a Scottish connection, and the Presbyterian denomination was also taken to North America mostly by Scots and Sc

In [15]:
print(chosen_contexts[26])

Raised in Chicago, West briefly attended art school before becoming known as a producer for Roc-A-Fella Records in the early 2000s, producing hit singles for artists such as Jay-Z and Alicia Keys. Intent on pursuing a solo career as a rapper, West released his debut album The College Dropout in 2004 to widespread commercial and critical success, and founded record label GOOD Music. He went on to explore a variety of different musical styles on subsequent albums that included the baroque-inflected Late Registration (2005), the arena-inspired Graduation (2007), and the starkly polarizing 808s & Heartbreak (2008). In 2010, he released his critically acclaimed fifth album, the maximalist My Beautiful Dark Twisted Fantasy, and the following year he collaborated with Jay-Z on the joint LP Watch the Throne (2011). West released his abrasive sixth album, Yeezus, to further critical praise in 2013. Following a series of recording delays and work on non-musical projects, West's seventh album, Th

In [16]:
df_melt[(df_melt.value > 0) & (df_melt.value < 1)].query('index != x').sort_values(by='value',ascending=False)

,index,x,value
725,5,24,0.222489
174,24,5,0.222489
169,19,5,0.155133
575,5,19,0.155133
760,10,25,0.141767
...,...,...,...
316,16,10,0.007601
430,10,14,0.007558
314,14,10,0.007558
312,12,10,0.007200


In [17]:

label_sets = []

for e in np.array([1,2,3,5,8,13])/10:
    for m in np.array([1,2,3,5,8,13,21,34,55,89,144]):
        #print(e)
        model_DB = DBSCAN(eps = e, min_samples = m, metric = 'euclidean').fit(csimDf)
        labels = model_DB.labels_
        centroids = derive_centroids(csimDf,labels)
        label_sets.append([e,m,len(np.unique(labels)),labels,centroids,bcss_tss_ratio(csimDf,centroids)])

C:\Users\User\AppData\Local\Temp\ipykernel_17008\1622011073.py:204: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\Temp\ipykernel_17008\1622011073.py:204: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\Temp\ipykernel_17008\1622011073.py:204: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\Temp\ipykernel_17008\1622011073.py:204: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\Temp\ipykernel_17008\1622011073.py:204: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\Temp\ipykernel_17008\1622011073.py:204: RuntimeWarning: Mean of empty slice
  centroids[i] = np.nanmean(data.iloc[indices], axis = 0)
C:\Users\User\AppData\Local\

In [18]:
results = pd.DataFrame(label_sets)
results.columns = ['e','m','clusters','labels','centroids','ratio']

In [19]:
candidate_clusters = results[results['clusters']<len(chosen_contexts)].query('ratio > 0').sort_values('clusters',ascending=True)

In [20]:
candidate_clusters

,e,m,clusters,labels,centroids,ratio
56,1.3,2,4,"[0, 0, -1, 1, 1, 2, 1, 0, -1, 1, 1, 1, 0, -1, -1, 0, 0, -1, 0, 2, -1, 1, 0, -1, 2, 1, 1, 1, 1, -1]","[[0.19114331130863377, 0.16993532251798563, 0.016701775014338047, 0.049253035159628256, 0.014089967659053154, 0.03370585603303945, 0.02302730535709927, 0.17348100299193703, 0.03735760659531596, 0.016093945747575528, 0.030239592432589828, 0.027855617124760562, 0.1905722134028993, 0.01824372608544495, 0.04394394730161256, 0.17508070016630922, 0.16701770402850544, 0.02487804264016327, 0.16074062001154787, 0.020910091075910645, 0.03220204996149216, 0.048563343731042194, 0.16164105491516262, 0.0017684452527510386, 0.03525986859665482, 0.029798588412262128, 0.021070930414606528, 0.05365722755707...",89.975523
57,1.3,3,4,"[0, 0, -1, 1, 1, 2, 1, 0, -1, 1, 1, 1, 0, -1, -1, 0, 0, -1, 0, 2, -1, 1, 0, -1, 2, 1, 1, 1, 1, -1]","[[0.19114331130863377, 0.16993532251798563, 0.016701775014338047, 0.049253035159628256, 0.014089967659053154, 0.03370585603303945, 0.02302730535709927, 0.17348100299193703, 0.03735760659531596, 0.016093945747575528, 0.030239592432589828, 0.027855617124760562, 0.1905722134028993, 0.01824372608544495, 0.04394394730161256, 0.17508070016630922, 0.16701770402850544, 0.02487804264016327, 0.16074062001154787, 0.020910091075910645, 0.03220204996149216, 0.048563343731042194, 0.16164105491516262, 0.0017684452527510386, 0.03525986859665482, 0.029798588412262128, 0.021070930414606528, 0.05365722755707...",89.975523
55,1.3,1,11,"[0, 0, 1, 2, 2, 3, 2, 0, 4, 2, 2, 2, 0, 5, 6, 0, 0, 7, 0, 3, 8, 2, 0, 9, 3, 2, 2, 2, 2, 10]","[[0.19114331130863377, 0.16993532251798563, 0.016701775014338047, 0.049253035159628256, 0.014089967659053154, 0.03370585603303945, 0.02302730535709927, 0.17348100299193703, 0.03735760659531596, 0.016093945747575528, 0.030239592432589828, 0.027855617124760562, 0.1905722134028993, 0.01824372608544495, 0.04394394730161256, 0.17508070016630922, 0.16701770402850544, 0.02487804264016327, 0.16074062001154787, 0.020910091075910645, 0.03220204996149216, 0.048563343731042194, 0.16164105491516262, 0.0017684452527510386, 0.03525986859665482, 0.029798588412262128, 0.021070930414606528, 0.05365722755707...",330.161114


In [22]:
for c in candidate_clusters.index:
    print("Candidate cluster:")
    temp_df = pd.DataFrame(chosen_contexts)
    temp_df['labels'] = candidate_clusters.loc[[c]][['labels']].values[0][0]
    labels = np.unique(temp_df['labels'])
    
    for l in labels[labels>=0]:
        subset = temp_df[temp_df['labels']==l]
        if(len(subset)==1):
            pass
        else:
            print(subset)
        #print(temp_df[temp_df['labels']>=0].sort_values(by='labels',ascending=False))

Candidate cluster:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          0  \
0   As federal judge Alex Kozinski has pointed out, binding precedent as we know it today simply did not exist at the time the Constitution was framed. Judicial decisions were not consistently, accurately, and faithfully reported on both sides of the Atlantic (reporters often simply rewrote or failed to publish decisions which they disliked), and the United Kingdom lacked